In [261]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline

In [262]:
df = pd.read_csv('datos/aguas_subterraneas_2020.csv')
df.head()

,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,ACUIFERO,SUBTIPO,LONGITUD,LATITUD,PERIODO,...,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
0,DLAGU6,POZO SAN GIL,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,ASIENTOS,VALLE DE CHICALOTE,POZO,-102.02210,22.20887,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
1,DLAGU6516,POZO R013 CA�ADA HONDA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,AGUASCALIENTES,VALLE DE CHICALOTE,POZO,-102.20075,21.99958,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
2,DLAGU7,POZO COSIO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,COSIO,VALLE DE AGUASCALIENTES,POZO,-102.28801,22.36685,2020,...,SI,SI,SI,NO,SI,SI,SI,SI,SI,SI
3,DLAGU9,POZO EL SALITRILLO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,VALLE DE AGUASCALIENTES,POZO,-102.29449,22.18435,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
4,DLBAJ107,RANCHO EL TECOLOTE,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,POZO,-110.24480,23.45138,2020,...,SI,SI,NO,SI,SI,SI,SI,SI,SI,SI


En el ejercicio anterior habíamos concluido que estaríamos usando las variables numéricas en lugar de las binarias y categóricas para los atributos químicos, a excepción del "Cadmio", ya que solo tienen 3 datos únicos y por tanto cuenta con casi la misma informaicón que la binaria.

También concluimos eliminar la variable "SDT_mg/L", puesto que la columna está completamente vacía, además que es un atributo que puede ser estimado a partir de la conductividad, por lo que ambas tendrían una correlación alta.

Por último, también se eliminan aquellas variables con una correlación mayor o igual a 0.75, como el caso del hierro, manganeso y mercurio. Por tanto, nos estaremos quedando solamente con la variable del hierro.

In [263]:
cols_to_drop  = df.columns[df.columns.str.contains('(?!CUMPLE_CON_CD)(?:CALIDAD|CUMPLE)')].to_list()
cols_to_drop.extend(['CLAVE', 'SITIO', 'CONTAMINANTES', 'ORGANISMO_DE_CUENCA', 'PERIODO', 
                     'CD_TOT_mg/L', 'ACUIFERO', 'SDT_mg/L', 'LONGITUD', 'LATITUD' , 'HG_TOT_mg/L', 'MN_TOT_mg/L'])

Definimos nuestras X y y

In [264]:

X = df.drop(cols_to_drop + ['SEMAFORO'], axis = 1)

In [265]:
X.head()

,ESTADO,MUNICIPIO,SUBTIPO,ALC_mg/L,CONDUCT_mS/cm,SDT_M_mg/L,FLUORUROS_mg/L,DUR_mg/L,COLI_FEC_NMP/100_mL,N_NO3_mg/L,AS_TOT_mg/L,CR_TOT_mg/L,PB_TOT_mg/L,FE_TOT_mg/L,CUMPLE_CON_CD
0,AGUASCALIENTES,ASIENTOS,POZO,229.990,940.0,603.6,0.9766,213.732,<1.1,4.184656,0.0161,<0.005,<0.005,0.0891,SI
1,AGUASCALIENTES,AGUASCALIENTES,POZO,231.990,608.0,445.4,0.9298,185.0514,<1.1,5.75011,0.0134,<0.005,<0.005,<0.025,SI
2,AGUASCALIENTES,COSIO,POZO,204.920,532.0,342,1.8045,120.719,<1.1,1.449803,0.037,<0.005,<0.005,<0.025,SI
3,AGUASCALIENTES,RINCON DE ROMOS,POZO,327.000,686.0,478.6,1.1229,199.879,<1.1,1.258597,0.0154,0.005,<0.005,<0.025,SI
4,BAJA CALIFORNIA SUR,LA PAZ,POZO,309.885,1841.0,1179,0.2343,476.9872,291,15.672251,<0.01,<0.005,<0.005,<0.025,SI


In [266]:
y = df[['SEMAFORO']]

In [267]:
y.head()

,SEMAFORO
0,Verde
1,Verde
2,Rojo
3,Verde
4,Rojo


Ahora clasificamos cada columna por su tipo de dato

In [268]:
bin_vars =  ['CUMPLE_CON_CD']
cat_vars = ['ESTADO', 'MUNICIPIO', 'SUBTIPO']
num_vars = X.drop(cat_vars + bin_vars, axis = 1).columns.to_list()

print(f'Numeric variables: {num_vars}\n')
print(f'Categorical variables: {cat_vars}\n')
print(f'Binary variables: {bin_vars}')

Numeric variables: ['ALC_mg/L', 'CONDUCT_mS/cm', 'SDT_M_mg/L', 'FLUORUROS_mg/L', 'DUR_mg/L', 'COLI_FEC_NMP/100_mL', 'N_NO3_mg/L', 'AS_TOT_mg/L', 'CR_TOT_mg/L', 'PB_TOT_mg/L', 'FE_TOT_mg/L']

Categorical variables: ['ESTADO', 'MUNICIPIO', 'SUBTIPO']

Binary variables: ['CUMPLE_CON_CD']


In [269]:
class TextToNumImputer(BaseEstimator, TransformerMixin):
    
    
    def get_text_values(self, df):
        vars = [
        'ALC_mg/L','AS_TOT_mg/L', 'CD_TOT_mg/L', 'FE_TOT_mg/L', 'SDT_M_mg/L',
        'SDT_mg/L', 'FLUORUROS_mg/L', 'COLI_FEC_NMP/100_mL', 'CONDUCT_mS/cm', 
        'CR_TOT_mg/L', 'DUR_mg/L', 'HG_TOT_mg/L', 'MN_TOT_mg/L', 'N_NO3_mg/L', 'PB_TOT_mg/L' ]

        limits = {}
        for  var in vars:
                if  (var in df.columns and df[var].dtypes == 'object'):
                        #Obtenemos las observaciones que solo tengan < > o =
                        limits[var] = df[df[var].str.contains('<|>|=', na='na') == True][var].value_counts().index
        #Lo convertimos en DF para mejor visualización
        return pd.DataFrame(limits, index = [0])
    
    def extract_values(self, df):
        #Extraemos las columnas que deben ser convertidas de object a float
        limits_df = self.get_text_values(df)

        #Hacemos un transpose para que la tabla tenga dos columnas, una con los atributos químicos y otra con sus valores
        limits_df = limits_df.transpose()
        limits_df = limits_df.reset_index(level = 0)
        limits_df.rename(columns={'index': 'attributes', 0:'attr_values'}, inplace= True)

        #Extraemos los valores numéricos, eliminando cualquier string (por ej. '<')
        limits_df.attr_values = limits_df.attr_values.str.extract('(\d+\.\d+|\d+)')

        return limits_df
   
    def replace_text(self, df):
        
        limits_df = self.extract_values(df)
                
        for col in limits_df.attributes:
            val =  (limits_df.loc[limits_df.attributes == col].attr_values.values[0])
            str_match = '<'+ val
            val = float(val)

            df.loc[df[col] == str_match, col] = val -(val/10)
            
            #Convertimos la columna a tipo float
            df[col] = df[col].astype(float)
            
        return df
        
    def fit(self, X):
        return self
    
    def transform(self, X):
        
        return self.replace_text(X)

In [270]:
cat_pipeline = Pipeline(steps = [
    ('cat-imp-mode', SimpleImputer(strategy='most_frequent')),
    ])
bin_pipeline = Pipeline(steps = [
    ('bin-imp-mode', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
    ])
num_pipeline = Pipeline(steps = [
    ('text-imputer', TextToNumImputer()),
    ('imp-median', SimpleImputer(strategy='median'))])

preprocessor = ColumnTransformer(transformers = [
    ('bin-imp', bin_pipeline, bin_vars),
    ('catimp', cat_pipeline, cat_vars),
    ('custom-imp', num_pipeline, (num_vars)),
   ],
    remainder='passthrough')
			

#probamos el transformer
scaled_f = preprocessor.fit_transform(X)

#Imprimimos una muestra de los datos transformados
scaled_f[:3,:]

array([[0.0, 1.0, 'AGUASCALIENTES', 'ASIENTOS', 'POZO', 229.99, 940.0,
        603.6, 0.9766, 213.732, 0.9900000000000001, 4.184656, 0.0161,
        0.0045000000000000005, 0.0045000000000000005, 0.0891],
       [0.0, 1.0, 'AGUASCALIENTES', 'AGUASCALIENTES', 'POZO', 231.99,
        608.0, 445.4, 0.9298, 185.0514, 0.9900000000000001, 5.75011,
        0.0134, 0.0045000000000000005, 0.0045000000000000005,
        0.022500000000000003],
       [0.0, 1.0, 'AGUASCALIENTES', 'COSIO', 'POZO', 204.92, 532.0,
        342.0, 1.8045, 120.719, 0.9900000000000001, 1.449803, 0.037,
        0.0045000000000000005, 0.0045000000000000005,
        0.022500000000000003]], dtype=object)